In [109]:
from pymongo import MongoClient
import pandas as pd
import pprint as pprint
import collections
from itertools import izip

In [14]:
client = MongoClient()
db = client.proj

In [286]:
cursor = db.listings.find({"address.state": "CA"}) # returns a generator

In [287]:
results = [document for document in cursor]
print len(results) # use this insead of cast as list because some documents are lost with list(cursor)

764


In [288]:
# taken from Stackoverflow discussion at:
# http://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [289]:
def rename_columns(df):
    cols = ['_id', 'pageviewcount_currentmonth', 'pageviewcount_total'] # want to keep these names as they are
    df.columns = [col[col.find('_')+1:] if col not in cols else col for col in df.columns]
    return df

In [295]:
def drop_columns(df):
    df = df.drop(['image', 'openhousedates', 'mls', 'lastupdateddate',
             'externalurl', 'agentprofileurl', 'agentname', 'homedetails', 'mls', 
             'homeinfo', 'photogallery'], axis=1)
    return df

In [281]:
# Find the indices of the rows that are repeated
def remove_duplicate_id(df):
    grouped = df.groupby('zpid').apply(len)
    repeated_rows = []
    for tupl in izip(grouped.index, grouped):
        if tupl[1] > 1:
            repeated_rows.append(tupl[0])
    t = df[df['zpid'].isin(repeated_rows)].sort('zpid')
    indices = []
    count = 0
    for row in t.iterrows():
        if count % 2 == 0:
            indices.append(row[0])
        count += 1
    return df[~(df.index.isin(indices))]
    

In [296]:
flattened = [flatten(result) for result in results]

In [297]:
df = pd.DataFrame(flattened, index=range(len(flattened)))

In [298]:
df = rename_columns(df)

In [299]:
df = drop_columns(df)

In [282]:
df = remove_duplicate_id(df)

In [300]:
df.shape

(764, 45)

In [304]:
df.to_csv('trx_sf.csv', separator=",")

In [305]:
ls

Untitled.ipynb        trx_sf.csv            web_scraping.ipynb    zillow_api.py
load_transform.ipynb  walkable_api.ipynb    zillow_api.ipynb      zillow_api.pyc


In [306]:
!mv trx_sf.csv ../data/san_fran/

In [307]:
ls

Untitled.ipynb        walkable_api.ipynb    zillow_api.ipynb      zillow_api.pyc
load_transform.ipynb  web_scraping.ipynb    zillow_api.py


In [308]:
cd ../data/san_fran/

/Users/michaelhood/Documents/projects/project/data/san_fran


In [309]:
ls

alameda.csv                   oakland.csv                   sf_protected_listings.csv
bernal_heights.csv            pacific_heights.csv           sf_protected_listings_v2.csv
daly_city.csv                 palo_alto.csv                 south_of_market.csv
menlo_park.csv                potrero_hill.csv              sunnyvale.csv
mission.csv                   redwood_city.csv              sunset_district.csv
mission_district_v2.csv       san_bruno.csv                 trx_sf.csv
nob_hill.csv                  santa_clara.csv               twin_peaks.csv


In [311]:
df.head()

,_id,city,latitude,longitude,state,street,zipcode,appliances,architecture,basement,...,middleschool,neighborhood,pageviewcount_currentmonth,pageviewcount_total,status,type,price,schooldistrict,whatownerloves,zpid
0,55f6e8a63ae7404c5edef1e6,Alameda,37.77089,-122.267093,CA,1046 Central Ave,94501,NaN,NaN,NaN,...,NaN,NaN,3490,3490,NaN,NaN,NaN,NaN,NaN,24862982
1,55f6e8a63ae7404c5edef1e7,Alameda,37.774675,-122.252674,CA,1724 Eagle Ave,94501,NaN,NaN,NaN,...,NaN,NaN,931,931,NaN,NaN,NaN,NaN,NaN,24861179
2,55f6e8a63ae7404c5edef1e8,Alameda,37.783418,-122.283656,CA,2378 Coral Sea St,94501,NaN,NaN,NaN,...,NaN,NaN,1372,1372,NaN,NaN,NaN,NaN,NaN,72700378
3,55f6e8a63ae7404c5edef1e9,Alameda,37.769562,-122.270395,CA,903 San Antonio Ave,94501,NaN,NaN,NaN,...,NaN,NaN,785,785,NaN,NaN,NaN,NaN,NaN,24863091
4,55f6e8a63ae7404c5edef1ea,Alameda,37.769149,-122.240074,CA,2318 Eagle Ave,94501,"Dishwasher, Dryer, Freezer, Garbage disposal, ...",Queen Anne / Victorian,NaN,...,NaN,NaN,1093,1093,NaN,Make me move,1395000,NaN,It's famous architecture.,24859684
